In [1]:
!pip install deepchem

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 13.9 MB/s eta 0:00:00


In [2]:
!pip install hyperopt

In [3]:
import deepchem as dc
tasks, datasets, transformers = dc.molnet.load_hiv(featurizer='ECFP', splitter = 'scaffold')
train_dataset, valid_dataset, test_dataset = datasets

Streaming output truncated to the last 5000 lines.
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:40:37] DEPRECATION WARNING: please use MorganGenerator
[21:4

In [4]:
from hyperopt import hp, fmin, tpe, Trials

In [5]:
search_space = {
    'layer_sizes': hp.choice('layer_sizes',[[500], [1000], [2000],[1000,1000]]),
    'dropouts': hp.uniform('dropout',low=0.2, high=0.5),
    'learning_rate': hp.uniform('learning_rate',high=0.001, low=0.0001)
}

In [6]:
search_space

{'layer_sizes': <hyperopt.pyll.base.Apply at 0x7fe98c21dad0>,
 'dropouts': <hyperopt.pyll.base.Apply at 0x7fe98bbee190>,
 'learning_rate': <hyperopt.pyll.base.Apply at 0x7fe98ba33610>}

In [7]:
import tempfile

In [8]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

In [9]:
def fm(args):
  save_dir = tempfile.mkdtemp()
  model = dc.models.MultitaskClassifier(n_tasks=len(tasks),n_features=1024,layer_sizes=args['layer_sizes'],dropouts=args['dropouts'],learning_rate=args['learning_rate'])
  #validation callback that saves the best checkpoint, i.e the one with the maximum score.
  validation=dc.models.ValidationCallback(valid_dataset, 1000, [metric],save_dir=save_dir,transformers=transformers,save_on_minimum=False)

  model.fit(train_dataset, nb_epoch=25,callbacks=validation)

  #restoring the best checkpoint and passing the negative of its validation score to be minimized.
  model.restore(model_dir=save_dir)
  valid_score = model.evaluate(valid_dataset, [metric], transformers)

  return -1*valid_score['roc_auc_score']

In [ ]:
trials=Trials()
best = fmin(fm,
    		space= search_space,
    		algo=tpe.suggest,
    		max_evals=15,
    		trials = trials)




  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]Step 1000 validation: roc_auc_score=0.737523
Step 2000 validation: roc_auc_score=0.769501
Step 3000 validation: roc_auc_score=0.774502
Step 4000 validation: roc_auc_score=0.784007
Step 5000 validation: roc_auc_score=0.777175
Step 6000 validation: roc_auc_score=0.767411
Step 7000 validation: roc_auc_score=0.774088
Step 8000 validation: roc_auc_score=0.769087


/usr/local/lib/python3.11/dist-packages/deepchem/models/torch_models/torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(checkpoint, map_loc

  7%|▋         | 1/15 [04:26<1:02:07, 266.25s/trial, best loss: -0.7840072769370711]Step 1000 validation: roc_auc_score=0.742712
Step 2000 validation: roc_auc_score=0.776217
Step 3000 validation: roc_auc_score=0.782814
Step 4000 validation: roc_auc_score=0.788803
Step 5000 validation: roc_auc_score=0.777553
Step 6000 validation: roc_auc_score=0.778372
Step 7000 validation: roc_auc_score=0.780282
Step 8000 validation: roc_auc_score=0.773897


/usr/local/lib/python3.11/dist-packages/deepchem/models/torch_models/torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(checkpoint, map_loc

 13%|█▎        | 2/15 [06:48<41:49, 193.07s/trial, best loss: -0.7888026743356273]  Step 1000 validation: roc_auc_score=0.756867
Step 2000 validation: roc_auc_score=0.776867
Step 3000 validation: roc_auc_score=0.751691
Step 4000 validation: roc_auc_score=0.765111
Step 5000 validation: roc_auc_score=0.747709
Step 6000 validation: roc_auc_score=0.746002
Step 7000 validation: roc_auc_score=0.744184
Step 8000 validation: roc_auc_score=0.739837


...

 87%|████████▋ | 13/15 [48:46<08:19, 249.67s/trial, best loss: -0.7888026743356273]Step 1000 validation: roc_auc_score=0.720734
Step 2000 validation: roc_auc_score=0.753348
Step 3000 validation: roc_auc_score=0.766806
Step 4000 validation: roc_auc_score=0.769152
Step 5000 validation: roc_auc_score=0.736942
Step 6000 validation: roc_auc_score=0.72992
Step 7000 validation: roc_auc_score=0.742834
Step 8000 validation: roc_auc_score=0.735178


/usr/local/lib/python3.11/dist-packages/deepchem/models/torch_models/torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(checkpoint, map_loc

 93%|█████████▎| 14/15 [53:58<04:28, 268.50s/trial, best loss: -0.7888026743356273]Step 1000 validation: roc_auc_score=0.732556
Step 2000 validation: roc_auc_score=0.755722
Step 3000 validation: roc_auc_score=0.768785
Step 4000 validation: roc_auc_score=0.767571
Step 5000 validation: roc_auc_score=0.7832
Step 6000 validation: roc_auc_score=0.773129
Step 7000 validation: roc_auc_score=0.771819
Step 8000 validation: roc_auc_score=0.766118


/usr/local/lib/python3.11/dist-packages/deepchem/models/torch_models/torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(checkpoint, map_loc

100%|██████████| 15/15 [56:17<00:00, 225.20s/trial, best loss: -0.7888026743356273]


In [ ]:
print("Best: {}".format(best))